In [1]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  



In [2]:
import configparser
config = configparser.ConfigParser()
config.read('config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')
window_size = int(config.get('Configuration', 'window_size'))

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

other_users=config.get('Configuration', 'other_users')
other_users = [(item.strip()) for item in other_users.split(',')]

users.extend(other_users)

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

other_anchors=config.get('Configuration', 'other_anchors')
other_anchors = [(item.strip()) for item in other_anchors.split(',')]

anchors.extend(other_anchors)

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]


room=config.get('Configuration', 'room')

In [3]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

days=list(range(day_time(t0),day_time(tN)+1))
print(days)

2024-03-16 21:31:18 Init date is 1710621078 day: 19798 2024-03-16 21:31:18
2024-03-16 21:41:04 End date is 1710621664 day: 19798 2024-03-16 21:41:04
[19798]


In [4]:
scene_user = {}
for d in days:
    for user in users:
        scene_user[user] = {}
        for s in scenes:
            print(d,source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv")
            try:
                series=read_csv(source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                print(d,series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                scene_user[user][s] = (series["time"].iloc[0],series["time"].iloc[-1])
                print(scene_user[user][s])
            except Exception as e:
                print("Not data",e)
                continue
def getScene(t0):
    for user, user_scene in scene_user.items():
        for scene, times in user_scene.items():
            print(t0,times[0],times[1])
            if(t0>times[0]-window_size/2 and t0<times[1]+window_size/2):
                return scene
    return None

19798 ./data/kitchen/2-person-labelled/Scene1/location_gt_ed9c/19798.location.tsv
19798 1215 (1710621088, 1710621359)
(1710621088, 1710621359)
19798 ./data/kitchen/2-person-labelled/Scene2/location_gt_ed9c/19798.location.tsv
19798 1380 (1710621360, 1710621664)
(1710621360, 1710621664)
19798 ./data/kitchen/2-person-labelled/Scene1/location_gt_867f/19798.location.tsv
19798 1145 (1710621078, 1710621359)
(1710621078, 1710621359)
19798 ./data/kitchen/2-person-labelled/Scene2/location_gt_867f/19798.location.tsv
19798 1255 (1710621362, 1710621664)
(1710621362, 1710621664)
19798 ./data/kitchen/2-person-labelled/Scene1/location_gt_f02e/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/kitchen/2-person-labelled/Scene1/location_gt_f02e/19798.location.tsv'
19798 ./data/kitchen/2-person-labelled/Scene2/location_gt_f02e/19798.location.tsv
Not data [Errno 2] No such file or directory: './data/kitchen/2-person-labelled/Scene2/location_gt_f02e/19798.location.tsv'


In [5]:
df_total_map = {}
for d in days:
    df_total_map[d]={}
    for user in users:
        df_total_map[d][user]=None
        for s in scenes:
            print(d,source_folder+"/"+s+"/ubitrack-uwb/"+str(d)+"."+user+".sourcedata.tsv")
            try:
                series=read_csv(source_folder+"/"+s+"/ubitrack-uwb/"+str(d)+"."+user+".sourcedata.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","anchor","d"])
                print(d,series.size)
            except:
                print("Not data")
                continue
            if(df_total_map[d][user] is None):
                df_total_map[d][user]=series
            else:
                df_total_map[d][user] = df_total_map[d][user].append(series, ignore_index=True)

            


19798 ./data/kitchen/2-person-labelled/Scene1/ubitrack-uwb/19798.ed9c.sourcedata.tsv
19798 1624
19798 ./data/kitchen/2-person-labelled/Scene2/ubitrack-uwb/19798.ed9c.sourcedata.tsv
19798 2908
19798 ./data/kitchen/2-person-labelled/Scene1/ubitrack-uwb/19798.867f.sourcedata.tsv
19798 2604
19798 ./data/kitchen/2-person-labelled/Scene2/ubitrack-uwb/19798.867f.sourcedata.tsv
19798 3232
19798 ./data/kitchen/2-person-labelled/Scene1/ubitrack-uwb/19798.f02e.sourcedata.tsv
19798 1276
19798 ./data/kitchen/2-person-labelled/Scene2/ubitrack-uwb/19798.f02e.sourcedata.tsv
19798 940


C:\Users\Usuario\AppData\Local\Temp\ipykernel_8096\4054099490.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][user] = df_total_map[d][user].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_8096\4054099490.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][user] = df_total_map[d][user].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_8096\4054099490.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][user] = df_total_map[d][user].append(series, ignore_index=True)


In [6]:
import cv2
import os

min_v=0
max_v=-100

for d in days:
    print(d)
    os.makedirs(data_folder+"/"+room+"/DAY_"+str(d)+"/", exist_ok=True)
    for user in users:
        # Check if the DataFrame exists for the day and user
        df_total = df_total_map.get(d, {}).get(user)
        if df_total is None:
            print(f"No data for day {d} and user {user}. Skipping.")
            continue
        for anchor in anchors:
            with open(data_folder+"/"+room+"/DAY_"+str(d)+"/"+user+"."+anchor+".tsv", "w") as filew:
                for tt in range(t0, tN, 1):
                    print(tt, day_time(tt), anchor)
                    if day_time(tt) != d:
                        continue
                    
                    df_intervalo = df_total[(df_total['time'] >= (tt-1)) & (df_total['time'] <= tt) & (df_total['anchor'] == anchor)]
                    if df_intervalo.size > 0:
                        v = np.min(df_intervalo['d'])
                        min_v = min(min_v, v)
                        max_v = max(max_v, v)
                        filew.write(f"{tt}\t{v}\n")
                    else:
                        filew.write(f"{tt}\t-1\n")



19798
1710621078 19798 1fa0
1710621079 19798 1fa0
1710621080 19798 1fa0
1710621081 19798 1fa0
1710621082 19798 1fa0
1710621083 19798 1fa0
1710621084 19798 1fa0
1710621085 19798 1fa0
1710621086 19798 1fa0
1710621087 19798 1fa0
1710621088 19798 1fa0
1710621089 19798 1fa0
1710621090 19798 1fa0
1710621091 19798 1fa0
1710621092 19798 1fa0
1710621093 19798 1fa0
1710621094 19798 1fa0
1710621095 19798 1fa0
1710621096 19798 1fa0
1710621097 19798 1fa0
1710621098 19798 1fa0
1710621099 19798 1fa0
1710621100 19798 1fa0
1710621101 19798 1fa0
1710621102 19798 1fa0
1710621103 19798 1fa0
1710621104 19798 1fa0
1710621105 19798 1fa0
1710621106 19798 1fa0
1710621107 19798 1fa0
1710621108 19798 1fa0
1710621109 19798 1fa0
1710621110 19798 1fa0
1710621111 19798 1fa0
1710621112 19798 1fa0
1710621113 19798 1fa0
1710621114 19798 1fa0
1710621115 19798 1fa0
1710621116 19798 1fa0
1710621117 19798 1fa0
1710621118 19798 1fa0
1710621119 19798 1fa0
1710621120 19798 1fa0
1710621121 19798 1fa0
1710621122 19798 1fa0
1710

1710621519 19798 1fa0
1710621520 19798 1fa0
1710621521 19798 1fa0
1710621522 19798 1fa0
1710621523 19798 1fa0
1710621524 19798 1fa0
1710621525 19798 1fa0
1710621526 19798 1fa0
1710621527 19798 1fa0
1710621528 19798 1fa0
1710621529 19798 1fa0
1710621530 19798 1fa0
1710621531 19798 1fa0
1710621532 19798 1fa0
1710621533 19798 1fa0
1710621534 19798 1fa0
1710621535 19798 1fa0
1710621536 19798 1fa0
1710621537 19798 1fa0
1710621538 19798 1fa0
1710621539 19798 1fa0
1710621540 19798 1fa0
1710621541 19798 1fa0
1710621542 19798 1fa0
1710621543 19798 1fa0
1710621544 19798 1fa0
1710621545 19798 1fa0
1710621546 19798 1fa0
1710621547 19798 1fa0
1710621548 19798 1fa0
1710621549 19798 1fa0
1710621550 19798 1fa0
1710621551 19798 1fa0
1710621552 19798 1fa0
1710621553 19798 1fa0
1710621554 19798 1fa0
1710621555 19798 1fa0
1710621556 19798 1fa0
1710621557 19798 1fa0
1710621558 19798 1fa0
1710621559 19798 1fa0
1710621560 19798 1fa0
1710621561 19798 1fa0
1710621562 19798 1fa0
1710621563 19798 1fa0
1710621564

1710621437 19798 a304
1710621438 19798 a304
1710621439 19798 a304
1710621440 19798 a304
1710621441 19798 a304
1710621442 19798 a304
1710621443 19798 a304
1710621444 19798 a304
1710621445 19798 a304
1710621446 19798 a304
1710621447 19798 a304
1710621448 19798 a304
1710621449 19798 a304
1710621450 19798 a304
1710621451 19798 a304
1710621452 19798 a304
1710621453 19798 a304
1710621454 19798 a304
1710621455 19798 a304
1710621456 19798 a304
1710621457 19798 a304
1710621458 19798 a304
1710621459 19798 a304
1710621460 19798 a304
1710621461 19798 a304
1710621462 19798 a304
1710621463 19798 a304
1710621464 19798 a304
1710621465 19798 a304
1710621466 19798 a304
1710621467 19798 a304
1710621468 19798 a304
1710621469 19798 a304
1710621470 19798 a304
1710621471 19798 a304
1710621472 19798 a304
1710621473 19798 a304
1710621474 19798 a304
1710621475 19798 a304
1710621476 19798 a304
1710621477 19798 a304
1710621478 19798 a304
1710621479 19798 a304
1710621480 19798 a304
1710621481 19798 a304
1710621482

1710621495 19798 6aca
1710621496 19798 6aca
1710621497 19798 6aca
1710621498 19798 6aca
1710621499 19798 6aca
1710621500 19798 6aca
1710621501 19798 6aca
1710621502 19798 6aca
1710621503 19798 6aca
1710621504 19798 6aca
1710621505 19798 6aca
1710621506 19798 6aca
1710621507 19798 6aca
1710621508 19798 6aca
1710621509 19798 6aca
1710621510 19798 6aca
1710621511 19798 6aca
1710621512 19798 6aca
1710621513 19798 6aca
1710621514 19798 6aca
1710621515 19798 6aca
1710621516 19798 6aca
1710621517 19798 6aca
1710621518 19798 6aca
1710621519 19798 6aca
1710621520 19798 6aca
1710621521 19798 6aca
1710621522 19798 6aca
1710621523 19798 6aca
1710621524 19798 6aca
1710621525 19798 6aca
1710621526 19798 6aca
1710621527 19798 6aca
1710621528 19798 6aca
1710621529 19798 6aca
1710621530 19798 6aca
1710621531 19798 6aca
1710621532 19798 6aca
1710621533 19798 6aca
1710621534 19798 6aca
1710621535 19798 6aca
1710621536 19798 6aca
1710621537 19798 6aca
1710621538 19798 6aca
1710621539 19798 6aca
1710621540

1710621540 19798 399a
1710621541 19798 399a
1710621542 19798 399a
1710621543 19798 399a
1710621544 19798 399a
1710621545 19798 399a
1710621546 19798 399a
1710621547 19798 399a
1710621548 19798 399a
1710621549 19798 399a
1710621550 19798 399a
1710621551 19798 399a
1710621552 19798 399a
1710621553 19798 399a
1710621554 19798 399a
1710621555 19798 399a
1710621556 19798 399a
1710621557 19798 399a
1710621558 19798 399a
1710621559 19798 399a
1710621560 19798 399a
1710621561 19798 399a
1710621562 19798 399a
1710621563 19798 399a
1710621564 19798 399a
1710621565 19798 399a
1710621566 19798 399a
1710621567 19798 399a
1710621568 19798 399a
1710621569 19798 399a
1710621570 19798 399a
1710621571 19798 399a
1710621572 19798 399a
1710621573 19798 399a
1710621574 19798 399a
1710621575 19798 399a
1710621576 19798 399a
1710621577 19798 399a
1710621578 19798 399a
1710621579 19798 399a
1710621580 19798 399a
1710621581 19798 399a
1710621582 19798 399a
1710621583 19798 399a
1710621584 19798 399a
1710621585

...

1710621193 19798 1fa0
1710621194 19798 1fa0
1710621195 19798 1fa0
1710621196 19798 1fa0
1710621197 19798 1fa0
1710621198 19798 1fa0
1710621199 19798 1fa0
1710621200 19798 1fa0
1710621201 19798 1fa0
1710621202 19798 1fa0
1710621203 19798 1fa0
1710621204 19798 1fa0
1710621205 19798 1fa0
1710621206 19798 1fa0
1710621207 19798 1fa0
1710621208 19798 1fa0
1710621209 19798 1fa0
1710621210 19798 1fa0
1710621211 19798 1fa0
1710621212 19798 1fa0
1710621213 19798 1fa0
1710621214 19798 1fa0
1710621215 19798 1fa0
1710621216 19798 1fa0
1710621217 19798 1fa0
1710621218 19798 1fa0
1710621219 19798 1fa0
1710621220 19798 1fa0
1710621221 19798 1fa0
1710621222 19798 1fa0
1710621223 19798 1fa0
1710621224 19798 1fa0
1710621225 19798 1fa0
1710621226 19798 1fa0
1710621227 19798 1fa0
1710621228 19798 1fa0
1710621229 19798 1fa0
1710621230 19798 1fa0
1710621231 19798 1fa0
1710621232 19798 1fa0
1710621233 19798 1fa0
1710621234 19798 1fa0
1710621235 19798 1fa0
1710621236 19798 1fa0
1710621237 19798 1fa0
1710621238

1710621357 19798 a304
1710621358 19798 a304
1710621359 19798 a304
1710621360 19798 a304
1710621361 19798 a304
1710621362 19798 a304
1710621363 19798 a304
1710621364 19798 a304
1710621365 19798 a304
1710621366 19798 a304
1710621367 19798 a304
1710621368 19798 a304
1710621369 19798 a304
1710621370 19798 a304
1710621371 19798 a304
1710621372 19798 a304
1710621373 19798 a304
1710621374 19798 a304
1710621375 19798 a304
1710621376 19798 a304
1710621377 19798 a304
1710621378 19798 a304
1710621379 19798 a304
1710621380 19798 a304
1710621381 19798 a304
1710621382 19798 a304
1710621383 19798 a304
1710621384 19798 a304
1710621385 19798 a304
1710621386 19798 a304
1710621387 19798 a304
1710621388 19798 a304
1710621389 19798 a304
1710621390 19798 a304
1710621391 19798 a304
1710621392 19798 a304
1710621393 19798 a304
1710621394 19798 a304
1710621395 19798 a304
1710621396 19798 a304
1710621397 19798 a304
1710621398 19798 a304
1710621399 19798 a304
1710621400 19798 a304
1710621401 19798 a304
1710621402

1710621585 19798 6aca
1710621586 19798 6aca
1710621587 19798 6aca
1710621588 19798 6aca
1710621589 19798 6aca
1710621590 19798 6aca
1710621591 19798 6aca
1710621592 19798 6aca
1710621593 19798 6aca
1710621594 19798 6aca
1710621595 19798 6aca
1710621596 19798 6aca
1710621597 19798 6aca
1710621598 19798 6aca
1710621599 19798 6aca
1710621600 19798 6aca
1710621601 19798 6aca
1710621602 19798 6aca
1710621603 19798 6aca
1710621604 19798 6aca
1710621605 19798 6aca
1710621606 19798 6aca
1710621607 19798 6aca
1710621608 19798 6aca
1710621609 19798 6aca
1710621610 19798 6aca
1710621611 19798 6aca
1710621612 19798 6aca
1710621613 19798 6aca
1710621614 19798 6aca
1710621615 19798 6aca
1710621616 19798 6aca
1710621617 19798 6aca
1710621618 19798 6aca
1710621619 19798 6aca
1710621620 19798 6aca
1710621621 19798 6aca
1710621622 19798 6aca
1710621623 19798 6aca
1710621624 19798 6aca
1710621625 19798 6aca
1710621626 19798 6aca
1710621627 19798 6aca
1710621628 19798 6aca
1710621629 19798 6aca
1710621630

1710621198 19798 1fa0
1710621199 19798 1fa0
1710621200 19798 1fa0
1710621201 19798 1fa0
1710621202 19798 1fa0
1710621203 19798 1fa0
1710621204 19798 1fa0
1710621205 19798 1fa0
1710621206 19798 1fa0
1710621207 19798 1fa0
1710621208 19798 1fa0
1710621209 19798 1fa0
1710621210 19798 1fa0
1710621211 19798 1fa0
1710621212 19798 1fa0
1710621213 19798 1fa0
1710621214 19798 1fa0
1710621215 19798 1fa0
1710621216 19798 1fa0
1710621217 19798 1fa0
1710621218 19798 1fa0
1710621219 19798 1fa0
1710621220 19798 1fa0
1710621221 19798 1fa0
1710621222 19798 1fa0
1710621223 19798 1fa0
1710621224 19798 1fa0
1710621225 19798 1fa0
1710621226 19798 1fa0
1710621227 19798 1fa0
1710621228 19798 1fa0
1710621229 19798 1fa0
1710621230 19798 1fa0
1710621231 19798 1fa0
1710621232 19798 1fa0
1710621233 19798 1fa0
1710621234 19798 1fa0
1710621235 19798 1fa0
1710621236 19798 1fa0
1710621237 19798 1fa0
1710621238 19798 1fa0
1710621239 19798 1fa0
1710621240 19798 1fa0
1710621241 19798 1fa0
1710621242 19798 1fa0
1710621243

1710621319 19798 c492
1710621320 19798 c492
1710621321 19798 c492
1710621322 19798 c492
1710621323 19798 c492
1710621324 19798 c492
1710621325 19798 c492
1710621326 19798 c492
1710621327 19798 c492
1710621328 19798 c492
1710621329 19798 c492
1710621330 19798 c492
1710621331 19798 c492
1710621332 19798 c492
1710621333 19798 c492
1710621334 19798 c492
1710621335 19798 c492
1710621336 19798 c492
1710621337 19798 c492
1710621338 19798 c492
1710621339 19798 c492
1710621340 19798 c492
1710621341 19798 c492
1710621342 19798 c492
1710621343 19798 c492
1710621344 19798 c492
1710621345 19798 c492
1710621346 19798 c492
1710621347 19798 c492
1710621348 19798 c492
1710621349 19798 c492
1710621350 19798 c492
1710621351 19798 c492
1710621352 19798 c492
1710621353 19798 c492
1710621354 19798 c492
1710621355 19798 c492
1710621356 19798 c492
1710621357 19798 c492
1710621358 19798 c492
1710621359 19798 c492
1710621360 19798 c492
1710621361 19798 c492
1710621362 19798 c492
1710621363 19798 c492
1710621364

In [7]:
print(min_v,max_v)

0 10.315
